In [3]:
import glob

In [6]:
import numpy
import pandas
import pulp

In [10]:
df_ls = []
for file in glob.glob('../Data/SalaryHistory/*.csv'):
    df_ls.append(pandas.read_csv(file, sep=';'))

In [11]:
pastsalaries = pandas.concat(df_ls)

In [12]:
pastsalaries.head()

,Date,GID,Pos,Name,Starter,DK Pts,DK Salary,Team,H/A,Oppt,Team Score,Oppt Score,Minutes,Stat line
0,20151116,4382,PG,"Knight, Brandon",1,78.00,"$7,000",pho,H,lal,120,101,40.78,30pt 10rb 15as 4st 2to 3trey 11-23fg 5-7ft
1,20151116,4079,PG,"Westbrook, Russell",1,72.00,"$12,200",okc,A,mem,114,122,38.53,40pt 4rb 14as 3st 8to 5trey 13-19fg 9-13ft
2,20151116,4523,PG,"Lillard, Damian",1,47.75,"$8,700",por,A,sas,80,93,41.97,27pt 7rb 6as 1st 1bl 3to 1trey 10-21fg 6-6ft
3,20151116,3979,PG,"Conley, Mike",1,46.50,"$6,000",mem,H,okc,122,114,31.38,22pt 4rb 9as 2st 4trey 8-13fg 2-3ft
4,20151116,4073,PG,"Rose, Derrick",1,41.50,"$5,900",chi,H,ind,96,95,33.22,23pt 2rb 6as 2st 1bl 2trey 9-18fg 3-4ft


In [13]:
pastsalaries = pastsalaries.dropna()

In [14]:
pastsalaries['DK Salary'] = pastsalaries['DK Salary'].astype(str).map(
    lambda ele: int(ele.replace('$', '').replace(',', '')))

In [15]:
specificDate = pastsalaries[pastsalaries['Date'] == 20160306]

In [17]:
specificDate.shape

(70, 14)

In [18]:
# initialize variables
playerPositions = specificDate['Pos'].tolist()
playerNames = specificDate['Name'].tolist()
playerTeams = tuple(specificDate['Team'].unique())
playerCosts = tuple(specificDate['DK Salary'])
playerPoints = tuple(specificDate['DK Pts'])
salaryCap = 50000
numOfPlayers = range(len(playerCosts))

In [20]:
# create problem
problem = pulp.LpProblem("Optimal Line-Up", pulp.LpMaximize)

In [21]:
# create variable to represent each player
playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")

In [22]:
# objective: maximize sum of player points
problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)

In [23]:
# constraint: each player can only be chosen at most once
for i in numOfPlayers:
    problem += pulp.lpSum(playerInLineup[i]) <= 1

In [24]:
# constraints: sum of player costs must be less than or equal to the salary cap
problem += pulp.lpSum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap

In [25]:
# constraint: teams must have 8 players
problem += pulp.lpSum(problem.variables()) == 8

In [26]:
# constraint: 1 <= number of PG <= Max 3
pointguards = tuple(specificDate['Pos'] == 'PG')
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) >= 1
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) <= 3

In [27]:
# constraint: 1 <= number of SG <= Max 3
shootguards = tuple(specificDate['Pos'] == 'SG')
problem += pulp.lpSum(playerInLineup[i] * shootguards[i] for i in numOfPlayers) >= 1
problem += pulp.lpSum(playerInLineup[i] * shootguards[i] for i in numOfPlayers) <= 3

In [28]:
# constraint: 1 <= number of SF <= Max 3
smallforward = tuple(specificDate['Pos'] == 'SF')
problem += pulp.lpSum(smallforward[i] * playerInLineup[i] for i in numOfPlayers) >= 1
problem += pulp.lpSum(smallforward[i] * playerInLineup[i] for i in numOfPlayers) <= 3

In [29]:
# constraint: 1 <= number of PF <= Max 3
powerfoward = tuple(specificDate['Pos'] == 'PF')
problem += pulp.lpSum(powerfoward[i] * playerInLineup[i] for i in numOfPlayers) >= 1
problem += pulp.lpSum(powerfoward[i] * playerInLineup[i] for i in numOfPlayers) <= 3

In [30]:
# constraint: 1 <= number of C <= Max 2
centers = tuple(specificDate['Pos'] == 'C')
problem += pulp.lpSum(playerInLineup[i] * centers[i] for i in numOfPlayers) <= 2
problem += pulp.lpSum(playerInLineup[i] * centers[i] for i in numOfPlayers) >= 1

In [31]:
# create variables for teams
teamsInLineup = pulp.LpVariable.dicts("Teams", [i for i in specificDate['Team']], 0, 1, cat="Binary")
teams = tuple(specificDate['Team'])

In [ ]:
# constraint: at least two different games must be chosen
for i in numOfPlayers:
    
problem += pulp.
lpSum(playerInLineup[i] * teamsInLineup[i] for i in numOfPlayers) >= 2

In [ ]:
# solve problem with the COIN CBC solver
solver = pulp.PULP_CBC_CMD(options=[],keepFiles = 0, msg=1, fracGap=0, maxSeconds = "100")
KnapsackProb.solve(solver)